In [1]:
import os
import numpy as np
from pathlib import Path
import pypianoroll as ppr
import pretty_midi
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt

## Import Data (phrase)

In [11]:
def pltDrumTrack(track, beat_resolution=96):
#     track = track.reshape((-1, 84))
    track = np.append(np.zeros((track.shape[0], 24)), track, axis=1)
    track = np.append(track, np.zeros((track.shape[0], 20)), axis=1)
    track = np.where(track == 1, 128, 0)
    track = Track(pianoroll=track)
    
    fig, axs = track.plot(
        xtick='beat',
        ytick='octave',
        yticklabel='number',
        beat_resolution=beat_resolution,
    )
    fig.set_size_inches(30,10)
    y = axs.set_ylim(34, 81) # C0 - C2
    plt.show()

In [12]:
def pltReducedDrumTrack(track, beat_resolution=12):
#     track = track.reshape((-1, 84))
#     track = np.append(np.zeros((track.shape[0], 24)), track, axis=1)
    track = np.append(track, np.zeros((track.shape[0], 119)), axis=1)
    track = np.where(track == 1, 128, 0)
    track = Track(pianoroll=track)
    
    fig, axs = track.plot(
        xtick='beat',
        yticklabel='number',
        beat_resolution=beat_resolution,
    )
    fig.set_size_inches(30,10)
    y = axs.set_ylim(0, 10) # C0 - C2
    y = axs.set_yticks(range(10))
    plt.show()

In [2]:
def checkDrumEmpty(track):
    compare = (track == np.zeros(track.shape))
    count = np.size(compare) - np.count_nonzero(compare)
    if count > 3:
        return True
    return False

In [3]:
DEFAULT_DRUM_TYPE_PITCHES = [
    # bass drum
    [36, 35],

    # snare drum
    [38, 27, 28, 31, 32, 33, 34, 37, 39, 40, 56, 65, 66, 75, 85],

    # closed hi-hat
    [42, 44, 54, 68, 69, 70, 71, 73, 78, 80],

    # open hi-hat
    [46, 67, 72, 74, 79, 81],

    # low tom
    [45, 29, 41, 61, 64, 84],

    # mid tom
    [48, 47, 60, 63, 77, 86, 87],

    # high tom
    [50, 30, 43, 62, 76, 83],

    # crash cymbal
    [49, 55, 57, 58],

    # ride cymbal
    [51, 52, 53, 59, 82]
]

In [4]:
_drum_map = dict(enumerate(DEFAULT_DRUM_TYPE_PITCHES))
_inverse_drum_map =  dict((pitch, index)
                                  for index, pitches in _drum_map.items()
                                  for pitch in pitches)

In [5]:
class MultiDrumOneHotEncoding():
    def __init__(self):
        self._drum_type_pitches = DEFAULT_DRUM_TYPE_PITCHES
        self._drum_map = dict(enumerate(DEFAULT_DRUM_TYPE_PITCHES))
        self._inverse_drum_map = dict((pitch, index) 
                                  for index, pitches in _drum_map.items()
                                  for pitch in pitches)

    def encode_drum(self, pitches_in):
        nonzero = np.where(pitches_in==1)[0] + 24
        ret = np.zeros(len(self._drum_type_pitches))
        for reduced, pitches in _drum_map.items():
            for p in pitches:
                if p in nonzero:
                    ret[reduced] = 1
                    break
        return ret
    
    def decode_drum(self, pitches_out):
        ret = np.zeros(84)
        for reduced, p in enumerate(pitches_out):
            if p == 1:
                ret[self._drum_type_pitches[reduced][0] - 24] = 1
        return ret
drum_encoding = MultiDrumOneHotEncoding()

In [6]:
PATH_HOME = str(Path.home())
PATH_TAGS = [
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Blues.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Country.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Electronic.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Folk.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Jazz.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Latin.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Metal.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_New-Age.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Pop.id', # 8
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Punk.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Rap.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Reggae.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_RnB.id',
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_Rock.id', # 13
    '/home/herman/lpd/id_lists/tagtraum/tagtraum_World.id',   
]
number_files = [19, 448, 352, 33, 127, 70, 112, 29, 1086, 12, 89, 53, 264, 1668, 10]
PATH_FILES = PATH_HOME + '/local_dir/salu133445/git/aicomposer/data/train_x.txt'
PATH = PATH_HOME + '/local_dir/salu133445/git/aicomposer/data/lpd_cleansed_stacked_pianoroll_11_tracks/'

In [54]:
for i in range(len(PATH_TAGS)):
    genre_i = i
    print('>>> ' + PATH_TAGS[genre_i][44:-3])
    p_genre = './data/all/train_x_drum_' + PATH_TAGS[genre_i][44:-3] + '.npy'
    train_x_phr_np = np.load(p_genre)
    print(train_x_phr_np.shape)
    
    print(train_x_drum_clean.shape)
    nz = np.nonzero(train_x_drum_clean)
    if nz[2].size > 0:
        print('min: {}, max: {}'.format(nz[2].min(), nz[2].max()))

    for i in range(84):
        places = nz[0][np.where(nz[2] == i)]
        print('[{}]{}: {}'.format(
            i,
            pretty_midi.note_number_to_drum_name(i + 24),
            places.shape[0]))

    check = [ checkDrumEmpty(train_x_phr_np[i]) for i in range(train_x_phr_np.shape[0])]
    train_x_drum_clean = train_x_phr_np[check]
    print(train_x_drum_clean.shape)
    
    train_x_drum_clean_reduced = np.zeros((
        train_x_drum_clean.shape[0],
        train_x_drum_clean.shape[1],
        len(DEFAULT_DRUM_TYPE_PITCHES),
    ))

    for bar_i, bar in enumerate(train_x_drum_clean):
        print ('converting...{}/{}'.format(bar_i + 1, train_x_drum_clean.shape[0]), end="\r")
        for beat_i, beat in enumerate(bar):
            train_x_drum_clean_reduced[bar_i][beat_i] = drum_encoding.encode_drum(beat)

    print()
    print(train_x_drum_clean_reduced.shape)
    np.save('./data/train_x_drum_reduced_' + PATH_TAGS[genre_i][44:-3] + '.npy', train_x_drum_clean_reduced)

>>> World
(561, 96, 84)
(39606, 96, 84)
min: 2, max: 72
[0]: 0
[1]: 0
[2]: 15
[3]: 0
[4]: 1601
[5]: 1
[6]: 0
[7]: 431
[8]: 8
[9]: 710
[10]: 122
[11]Acoustic Bass Drum: 55218
[12]Bass Drum 1: 52069
[13]Side Stick: 19826
[14]Acoustic Snare: 39907
[15]Hand Clap: 7052
[16]Electric Snare: 28809
[17]Low Floor Tom: 2087
[18]Closed Hi Hat: 205942
[19]High Floor Tom: 2280
[20]Pedal Hi Hat: 24141
[21]Low Tom: 3808
[22]Open Hi Hat: 13137
[23]Low-Mid Tom: 2037
[24]Hi-Mid Tom: 1469
[25]Crash Cymbal 1: 4551
[26]High Tom: 515
[27]Ride Cymbal 1: 27253
[28]Chinese Cymbal: 729
[29]Ride Bell: 3084
[30]Tambourine: 30375
[31]Splash Cymbal: 1351
[32]Cowbell: 2136
[33]Crash Cymbal 2: 2377
[34]Vibraslap: 1
[35]Ride Cymbal 2: 9610
[36]Hi Bongo: 651
[37]Low Bongo: 1158
[38]Mute Hi Conga: 2913
[39]Open Hi Conga: 3585
[40]Low Conga: 2654
[41]High Timbale: 139
[42]Low Timbale: 24
[43]High Agogo: 202
[44]Low Agogo: 52
[45]Cabasa: 20589
[46]Maracas: 11030
[47]Short Whistle: 0
[48]Long Whistle: 0
[49]Short Guiro: 138